# Deriving impact parameter for QSO relative to galaxies

This notebook walks through calculating physical impact parameters from QSO and galaxy coordinates and galaxy redshifts.

We will use the case of two galaxies near to a QSO observed by HST/COS:

* **QSO**: SDSS J222836.32-095008.9 (z=0.797)
* **Gal 1**: SDSS J222844.17-094809.4 (z=0.0538)
* **Gal 2**: SDSS J222834.38-094943.4 (z~0.165)
* **Gal 3**: SDSS J222834.93-094939.9 (z~0.409)

In [1]:
# Typical imports here
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c

```

```
# Calculating angular offsets with `astropy.coordinates`

The first thing to do is calculate the angular offsets of each of the galaxies from the QSO.

In [2]:
from astropy.coordinates import SkyCoord

## 1. Define the QSO coordinates

In [4]:
qsoCoord = SkyCoord('22:28:36.31','-09:50:08.9',unit=('hourangle','deg'),frame='icrs')

In [6]:
qsoCoord.galactic

<SkyCoord (Galactic): (l, b) in deg
    (53.23685506, -52.18006454)>

We could have queried SIMBAD for the coordinates:

In [7]:
from astroquery.simbad import Simbad
# Create custom search
customSimbad = Simbad()

# Use alternative name for this QSO
simbad_data = customSimbad.query_object('SDSS J222836.32-095008.9')
qsoCoord = SkyCoord(simbad_data['RA'],simbad_data['DEC'],
                    unit=('hourangle','deg'),frame='icrs')

```

```
## 2. Define the galaxy coordinates

In [9]:
gal_ra = ['22:28:44.17','22:28:34.38','22:28:34.9']
gal_dec = ['-09:48:09.4','-09:49:43.4','-09:49:39.9']
galCoord = SkyCoord(gal_ra,gal_dec,unit=('hourangle','deg'),frame='icrs')

```

```
## 3. Calculate the angular separation of each galaxy from the QSO

In [10]:
angular_offsets = qsoCoord.separation(galCoord)

The offsets are stored as coordinates, with appropriate `astropy` units:

In [11]:
angular_offsets

<Angle [0.04624753, 0.01070344, 0.00998375] deg>

In [12]:
angular_offsets.to('arcsec')

<Angle [166.49110013,  38.53239509,  35.94149616] arcsec>

In [13]:
angular_offsets.to('arcmin')

<Angle [2.77485167, 0.64220658, 0.59902494] arcmin>

# Calculate physical impact parameters using `astropy.cosmology`

We are interested in transforming the angular offsets to physical offsets – the arclength. For cosmological distances, this requires using the _angular diameter distance_:

$r_\perp = D_A \theta$.

## 1.Set up our cosmology.

In [14]:
from astropy.cosmology import Planck15 as cosmo

The `cosmo` procedure now contains the functions needed to calculate cosmological distances, timescales, etc.

In [15]:
cosmo

FlatLambdaCDM(name="Planck15", H0=67.7 km / (Mpc s), Om0=0.307, Tcmb0=2.725 K, Neff=3.05, m_nu=[0.   0.   0.06] eV, Ob0=0.0486)

Rather than use a cosmology based on the Planck results, it's sometimes preferable to adopt a **"737" cosmology**:

In [16]:
from astropy.cosmology import FlatLambdaCDM

cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K,
                      Om0=0.3, Ob0=0.045)

H0 = 70.*u.km/u.s/u.Mpc # Hubble constant

In [17]:
cosmo

FlatLambdaCDM(H0=70 km / (Mpc s), Om0=0.3, Tcmb0=2.725 K, Neff=3.04, m_nu=[0. 0. 0.] eV, Ob0=0.045)

## 2(a). Calculate angular diameter distance to the three galaxies, apply this to calculate arclength

#### First calculate the angular diameter distance to each galaxy

In [18]:
zgal = np.array([0.0538, 0.165, 0.409])
DA = cosmo.angular_diameter_distance(zgal)

In [19]:
DA

<Quantity [ 215.97197486,  583.39222125, 1123.23365703] Mpc>

#### Calculate physical separation for each of the galaxies using $D_A$

In [20]:
# Notice the transformation to distances in kpc and angle in radians:
r_perp = DA.to('kpc')*angular_offsets.to('rad')/u.rad

This produces physical offsets in _proper_ distance units. 

In [21]:
r_perp

<Quantity [174.32645125, 108.98368931, 195.72266792] kpc>

## 2(b). Calculate physical separation for each of the galaxies using `cosmo` code

While we can calculate the results using the code specifically designed for this problem within `astropy.cosmology`.

In [24]:
scale = cosmo.kpc_proper_per_arcmin(zgal)
r_perp = scale*angular_offsets.to('arcmin')

In [25]:
scale

<Quantity [ 62.82370089, 169.70191819, 326.73542641] kpc / arcmin>

In [26]:
r_perp

<Quantity [174.32645125, 108.98368931, 195.72266792] kpc>

If we prefer _comoving separations_, we can do that easily, as well:

In [27]:
scale = cosmo.kpc_comoving_per_arcmin(zgal)
r_perp = scale*angular_offsets.to('arcmin')

In [28]:
r_perp

<Quantity [183.70521433, 126.96599804, 275.7732391 ] kpc>

Remember that comoving distances are just the proper distances $\times\ (1+z)$

In [35]:
r_perp / (1+zgal)

<Quantity [174.32645125, 108.98368931, 195.72266792] kpc>